In [ ]:
import deblend
import om10
import desc.slrealizer
import warnings
warnings.filterwarnings("ignore")

In [ ]:
db = om10.DB()
db.select_random(maglim=23.3,area=18000.0,IQ=0.75)
db.paint(synthetic=True)
realizer = desc.slrealizer.SLRealizer(catalog=db, observation="../../../data/twinkles_observation_history.csv")

In [ ]:
%matplotlib inline
%pylab inline
matplotlib.use('TkAgg')
matplotlib.rc('text', usetex=False)

In [ ]:
# Choose a good lens system, with large imsep and 4 images:
                      
wide = db.sample[np.where(db.sample['IMSEP'] > 1.0)]
print "Number of lenses with image separation > 1.0 arcsec = ",len(wide)

wide_quads = wide[np.where(wide['NIMG'] == 4)]
print "Number of these with 4 images = ",len(wide_quads)

bright_wide_quads = wide[np.where(wide_quads['MAGI'] < 22)]
print "Number of these with i3 < 22 = ",len(bright_wide_quads)

db.sample = bright_wide_quads    
                      
k = db.sample[0]['LENSID']

In [ ]:
realizer.test_covariance_matrix(lensID=k)

In [ ]:
realizer.deblend(lensID=k, null_deblend=True, debug=False)

In [ ]:
realizer.deblend(lensID=k, null_deblend=False, debug=True)

In [ ]:
realizer.deblend_distance(lensID=k, null_deblend=True, debug=True, show_plot=True)

### Below is the code that I should eventually delete

In [ ]:
realizer.deblend(lensID=k, null_deblend=False, debug=True)

In [ ]:
realizer.plot_lens_random_date(lensID=k)

In [ ]:
#deblender sample
from scipy.stats import multivariate_normal
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
import scipy.optimize as opt
%matplotlib inline
%pylab inline
matplotlib.use('TkAgg')
matplotlib.rc('text', usetex=False)

number_of_rows = 1000
number_of_columns = 1000
image = [[0]*number_of_rows for _ in range(number_of_columns)]
image1 = [[0]*number_of_rows for _ in range(number_of_columns)]

x, y = np.mgrid[-5:5:.01, -5:5:.01]
pos = np.dstack((x, y))

init_guess = []

init_guess.append([3, 1])
init_guess.append([2, 2])
init_guess.append([1, 1])

init_guess_sigma = []


print init_guess

rv = scipy.stats.multivariate_normal([3, 1]) # xPos, yPos, covariance matrix whose entrees are just FWHM
#print rv
image = image + rv.pdf(pos) # BUG: multiply flux ratio
# save peak value
#rv = scipy.stats.multivariate_normal([2, 2])
#image = image + rv.pdf(pos)
rv = scipy.stats.multivariate_normal([1, 1])
image = image + rv.pdf(pos)
#print image # this is a 2d array
#print image

cmap2 = matplotlib.colors.LinearSegmentedColormap.from_list('my_colormap', ['black', 'green', 'yellow'], 256)
img2 = plt.imshow(image, interpolation='nearest', cmap = cmap2, origin='lower', extent=[-3, 3, -3, 3], aspect = "auto")

plt.colorbar(img2,cmap=cmap2)
plt.show()

def twoD_Gaussian((x, y), amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    xo = float(xo)
    yo = float(yo)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return g

In [ ]:
import photutils
segm = photutils.detect_sources(image, 0.1, 3)
# plot the image and the segmentation image
import matplotlib.pyplot as plt
numpy.set_printoptions(threshold='nan')
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8))
ax2.imshow(segm.data, origin='lower', interpolation='nearest')
#print segm.data

#segm_deblend = photutils.segmentation.deblend_sources(image, segm.data, 4)
#plt.imshow(segm_deblend.data, origin='lower', interpolation='nearest')

In [ ]:
from photutils import DAOStarFinder

daofind = DAOStarFinder(fwhm=1.5, threshold=0.00001)
sources = daofind(image)
print sources

from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils import CircularAperture
positions = (sources['xcentroid'], sources['ycentroid'])
apertures = CircularAperture(positions, r=4.)
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(image, cmap='Greys', origin='lower', norm=norm)
apertures.plot(color='red', lw=3.0, alpha=1.0)

#from astropy.visualization import SqrtStretch
#from astropy.visualization.mpl_normalize import ImageNormalize
#from photutils import CircularAperture

#positions = (sources['xcentroid'], sources['ycentroid'])
#apertures = CircularAperture(positions, r=500)
#norm = ImageNormalize(stretch=SqrtStretch())
#img2 = plt.imshow(image, interpolation='nearest', cmap = cmap2, origin='lower', extent=[-3, 3, -3, 3], aspect = "auto")
#apertures.plot(color='white', lw=10, alpha=1)

In [ ]:
#deblend fit gaussian
#pick peaks out and then start fitting
#https://stackoverflow.com/questions/21566379/fitting-a-2d-gaussian-function-using-scipy-optimize-curve-fit-valueerror-and-m